In [ ]:
### GAMMEL KODE DER SKAL RENGØRES/SLETTES DET DER IKKE BRUGES 
# for at finde flere chessboards på et billede, laver vi bounding boxes omkring hvert chessboard således at 
# hver roi kun har ét chessboard i sig, som vi så kan køre findChessboardCornersSB på 

# json med rois
with open("rois_02.json", "r") as f:
    rois_02 = json.load(f)[:13]

with open("rois_03.json", "r") as f:
    rois_03 = json.load(f)[:13]


# Define chessboard patterns
patterns = [
    {"pattern_size": (11, 7), "square_size": 0.010},   # C
    {"pattern_size": (7, 5),  "square_size": 0.010},   # A
    {"pattern_size": (5, 7),  "square_size": 0.010},   # B
    {"pattern_size": (5, 15), "square_size": 0.010},   # D
]

# Termination criteria for cornerSubPix
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

path_02 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_02/data/*.png"
path_03 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_03/data/*.png"

images_02 = sorted(glob.glob(path_02))
images_03 = sorted(glob.glob(path_03))

print("02 images:", len(images_02))
print("03 images:", len(images_03))

objpoints = []               # 3D real world positions 
imgpoints_image_02 = []      # 2D detected points in images 
imgpoints_image_03 = []      # 2D detected points in images 

# looper over alle 19 par med 1 billede fra 02, og 1 billede fra 03. Tilføjer alle par til calibration, selv hvis 
#hvis ikke alle chessboards er fundet, because thats life 
for f02, f03 in zip(images_02, images_03):
    print(f"\nProcessing pair: {f02} + {f03}")

    gray02 = cv2.cvtColor(cv2.imread(f02), cv2.COLOR_BGR2GRAY)
    gray03 = cv2.cvtColor(cv2.imread(f03), cv2.COLOR_BGR2GRAY)

    corners_pair_02 = []
    corners_pair_03 = []
    objp_pair = []

    for idx, (r02, r03) in enumerate(zip(rois_02, rois_03)):
        x1, y1, x2, y2 = r02
        x1b, y1b, x2b, y2b = r03

        # roi koordinater på billederne 
        h02, w02 = gray02.shape
        h03, w03 = gray03.shape
        x1, x2 = max(0, min(x1, w02-1)), max(0, min(x2, w02-1))
        y1, y2 = max(0, min(y1, h02-1)), max(0, min(y2, h02-1))
        x1b, x2b = max(0, min(x1b, w03-1)), max(0, min(x2b, w03-1))
        y1b, y2b = max(0, min(y1b, h03-1)), max(0, min(y2b, h03-1))

        # Skippper invalid roi
        if x2 <= x1 or y2 <= y1 or x2b <= x1b or y2b <= y1b:
            print(f"  ⚠ Invalid ROI for board {idx+1}, skipping")
            continue

        crop02 = gray02[y1:y2, x1:x2]
        crop03 = gray03[y1b:y2b, x1b:x2b]

        board_found = False
        for pat in patterns:
            ps = pat["pattern_size"]
            sq = pat["square_size"]

            ok02, c02 = cv2.findChessboardCornersSB(crop02, ps) #finder chessboard corners indeni hver roi for hvert billede 
            ok03, c03 = cv2.findChessboardCornersSB(crop03, ps)

            if ok02 and ok03:
                print(f"  Board {idx+1}: pattern {ps} detected")

                c02 = cv2.cornerSubPix(crop02, c02, (11,11), (-1,-1), criteria)
                c03 = cv2.cornerSubPix(crop03, c03, (11,11), (-1,-1), criteria)

                c02_full = c02 + np.array([[x1, y1]])
                c03_full = c03 + np.array([[x1b, y1b]])

                corners_pair_02.append(c02_full)
                corners_pair_03.append(c03_full)

                objp = np.zeros((ps[0]*ps[1], 3), np.float32)
                objp[:, :2] = np.mgrid[0:ps[0], 0:ps[1]].T.reshape(-1, 2)
                objp *= sq

                objp_pair.append(objp)
                board_found = True
                break

        if not board_found:
            print(f"  ⚠ Board {idx+1} NOT FOUND — skipping")

    # gemmer til calibration selvom det er flawed like hell 
    objpoints.extend(objp_pair)
    imgpoints_image_02.extend(corners_pair_02)
    imgpoints_image_03.extend(corners_pair_03)
    print(f"  ℹ Pair added for calibration ({len(corners_pair_02)} boards detected)")


In [ ]:


# json med rois
with open("rois_02.json", "r") as f:
    rois_02 = json.load(f)[:13]

with open("rois_03.json", "r") as f:
    rois_03 = json.load(f)[:13]


patterns = [
    {"pattern_size": (11, 7), "square_size": 0.010},   # C
    {"pattern_size": (7, 5),  "square_size": 0.010},   # A
    {"pattern_size": (5, 7),  "square_size": 0.010},   # B
    {"pattern_size": (5, 15), "square_size": 0.010},   # D
]

# Map each ROI index to a fixed pattern
roi_patterns = [
    0,  # ROI 1  → pattern index 0 (11x7)  C
    1,  # ROI 2  → pattern index 1 (7x5)   A
    2,  # ROI 3  → pattern index 2 (5x7)   B
    3,  # ROI 4  → pattern index 3 (5x15)  D
    # repeat however needed for your 13 ROIs
    0,1,2,3,0,1,2,3,0  # just an example — adjust as appropriate
]

objpoints   = []
imgpoints_02 = []
imgpoints_03 = []

for f02, f03 in zip(images_02, images_03):

    gray02 = cv2.cvtColor(cv2.imread(f02), cv2.COLOR_BGR2GRAY)
    gray03 = cv2.cvtColor(cv2.imread(f03), cv2.COLOR_BGR2GRAY)

    corners02_all = []
    corners03_all = []
    objp_all      = []

    for idx, (roi02, roi03) in enumerate(zip(rois_02, rois_03)):

        pat = patterns[ roi_patterns[idx] ]
        ps = pat["pattern_size"]
        sq = pat["square_size"]

        # extract roi
        x1,y1,x2,y2   = roi02
        x1b,y1b,x2b,y2b = roi03

        crop02 = gray02[y1:y2, x1:x2]
        crop03 = gray03[y1b:y2b, x1b:x2b]

        # try detect board in both cameras
        ok02, c02 = cv2.findChessboardCornersSB(crop02, ps)
        ok03, c03 = cv2.findChessboardCornersSB(crop03, ps)

        if not (ok02 and ok03):
            continue  # skip this board for this image

        # refine
        c02 = cv2.cornerSubPix(crop02, c02, (11,11), (-1,-1), criteria)
        c03 = cv2.cornerSubPix(crop03, c03, (11,11), (-1,-1), criteria)

        # shift to full-image coordinates
        c02_full = c02 + np.array([[x1, y1]])
        c03_full = c03 + np.array([[x1b, y1b]])

        # append
        #corners02_all.append(c02_full.reshape(-1,2))
        #corners03_all.append(c03_full.reshape(-1,2))

        corners02_all.append(c02_full.reshape(-1,2).astype(np.float32))
        corners03_all.append(c03_full.reshape(-1,2).astype(np.float32))

        objp = np.zeros((ps[0]*ps[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0:ps[0], 0:ps[1]].T.reshape(-1, 2)
        objp *= sq
        #objp_all.append(objp)
        objp_all.append(objp.astype(np.float32))

    # only add the image if BOTH cameras saw the same boards
    if len(corners02_all) > 0 and len(corners02_all) == len(corners03_all):

        imgpoints_02.append(np.vstack(corners02_all))
        imgpoints_03.append(np.vstack(corners03_all))
        objpoints.append(np.vstack(objp_all))


In [ ]:
#kalibrering af begge kameraer 

ret02, mtx02, dist02, rvecs02, tvecs02 = cv2.calibrateCamera(objpoints, imgpoints_02, gray02.shape[::-1], None, None)
ret03, mtx03, dist03, rvecs03, tvecs03 = cv2.calibrateCamera(objpoints, imgpoints_03, gray03.shape[::-1], None, None)


In [ ]:
# image size
image_size = (gray02.shape[1], gray02.shape[0])
print("Image size:", image_size)

# criteria for stereoCalibrate
criteria_stereo = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)

# Filter image pairs: only keep those where both cameras saw the same number of points
objpoints_filtered = []
imgpoints_02_filtered = []
imgpoints_03_filtered = []

for objp_img, pts02, pts03 in zip(objpoints, imgpoints_02, imgpoints_03):
    if pts02.shape[0] == pts03.shape[0] == objp_img.shape[0]:
        objpoints_filtered.append(objp_img)
        imgpoints_02_filtered.append(pts02)
        imgpoints_03_filtered.append(pts03)
    else:
        print(f"⚠ Skipping image due to mismatched points: {pts02.shape[0]}, {pts03.shape[0]}, {objp_img.shape[0]}")

# Make sure there are images left
if len(objpoints_filtered) == 0:
    raise RuntimeError("No valid image pairs found for stereo calibration!")

# Stereo calibration (using the intrinsics you already computed)
ret, mtx02, dist02, mtx03, dist03, R, T, E, F = cv2.stereoCalibrate(
    objpoints_filtered,
    imgpoints_02_filtered,
    imgpoints_03_filtered,
    mtx02, dist02,
    mtx03, dist03,
    image_size,
    criteria=criteria_stereo,
    flags=cv2.CALIB_FIX_INTRINSIC
)

print("\nStereo calibration results:")
print("Rotation R:\n", R)
print("Translation T:\n", T)


In [ ]:
#stereo calibration, finder camera extrinsics (rotation R og translation T)

image_size = (gray02.shape[1], gray02.shape[0])  # (width, height)
print("\nImage size:", image_size)

criteria_stereo = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS,
                   100,
                   1e-5)

# bliver nødt til at flatte points for at det passer til stereoCalibrate, lidt ligesom med cameraCalibrate
imgpoints_02 = [pts.reshape(-1,2).astype(np.float32) for pts in imgpoints_image_02]
imgpoints_03 = [pts.reshape(-1,2).astype(np.float32) for pts in imgpoints_image_03]
objpoints_stereo = [pts.astype(np.float32) for pts in objpoints]  # ensure float32


#nu bliver vi nødt til at filter efter par hvor boardsne er detected i begge billeder, ellers kan vi ikke stereoCalibrate, 
# da det er vigtigt at kameraerne self ser på det samme, hvis du forstår. 

objpoints_filtered = []
imgpoints_02_filtered = []
imgpoints_03_filtered = []

for objp_img, pts02, pts03 in zip(objpoints, imgpoints_image_02, imgpoints_image_03):
    if pts02.shape[0] == pts03.shape[0] == objp_img.shape[0]:
        objpoints_filtered.append(objp_img.astype(np.float32))
        imgpoints_02_filtered.append(pts02.reshape(-1,2).astype(np.float32))
        imgpoints_03_filtered.append(pts03.reshape(-1,2).astype(np.float32))
    else:
        print(f"⚠ Skipping image due to mismatched points: {pts02.shape[0]}, {pts03.shape[0]}, {objp_img.shape[0]}")


ret, mtx02, dist02, mtx03, dist03, R, T, E, F = cv2.stereoCalibrate(
    objpoints_filtered,
    imgpoints_02_filtered,
    imgpoints_03_filtered,
    mtx02, dist02,
    mtx03, dist03,
    image_size,
    criteria=criteria_stereo,
    flags=cv2.CALIB_FIX_INTRINSIC
)


print("\nStereo calibration resultater:")
print("Rotation:\n", R)
print("Translation:\n", T)



In [ ]:

patterns = [
    {"pattern_size": (11, 7), "square_size": 0.010},   # C
    {"pattern_size": (7, 5),  "square_size": 0.010},   # A
    {"pattern_size": (5, 7),  "square_size": 0.010},   # B
    {"pattern_size": (5, 15), "square_size": 0.010},   # D
]

# Map each ROI index to a fixed pattern
roi_patterns = [
    0,  # ROI 1  → pattern index 0 (11x7)  C
    1,  # ROI 2  → pattern index 1 (7x5)   A
    2,  # ROI 3  → pattern index 2 (5x7)   B
    3,  # ROI 4  → pattern index 3 (5x15)  D
    # repeat however needed for your 13 ROIs
    0,1,2,3,0,1,2,3,0  # just an example — adjust as appropriate
]

objpoints   = []
imgpoints_02 = []
imgpoints_03 = []

for f02, f03 in zip(images_02, images_03):

    gray02 = cv2.cvtColor(cv2.imread(f02), cv2.COLOR_BGR2GRAY)
    gray03 = cv2.cvtColor(cv2.imread(f03), cv2.COLOR_BGR2GRAY)

    corners02_all = []
    corners03_all = []
    objp_all      = []

    for idx, (roi02, roi03) in enumerate(zip(rois_02, rois_03)):

        pat = patterns[ roi_patterns[idx] ]
        ps = pat["pattern_size"]
        sq = pat["square_size"]

        # extract roi
        x1,y1,x2,y2   = roi02
        x1b,y1b,x2b,y2b = roi03

        crop02 = gray02[y1:y2, x1:x2]
        crop03 = gray03[y1b:y2b, x1b:x2b]

        # try detect board in both cameras
        ok02, c02 = cv2.findChessboardCornersSB(crop02, ps)
        ok03, c03 = cv2.findChessboardCornersSB(crop03, ps)

        if not (ok02 and ok03):
            continue  # skip this board for this image

        # refine
        c02 = cv2.cornerSubPix(crop02, c02, (11,11), (-1,-1), criteria)
        c03 = cv2.cornerSubPix(crop03, c03, (11,11), (-1,-1), criteria)

        # shift to full-image coordinates
        c02_full = c02 + np.array([[x1, y1]])
        c03_full = c03 + np.array([[x1b, y1b]])

        # append
        corners02_all.append(c02_full.reshape(-1,2))
        corners03_all.append(c03_full.reshape(-1,2))

        objp = np.zeros((ps[0]*ps[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0:ps[0], 0:ps[1]].T.reshape(-1, 2)
        objp *= sq
        objp_all.append(objp)

    # only add the image if BOTH cameras saw the same boards
    if len(corners02_all) > 0 and len(corners02_all) == len(corners03_all):

        imgpoints_02.append(np.vstack(corners02_all))
        imgpoints_03.append(np.vstack(corners03_all))
        objpoints.append(np.vstack(objp_all))


In [ ]:
#stereo rectification 

R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    cameraMatrix1=mtx02,
    distCoeffs1=dist02,
    cameraMatrix2=mtx03,
    distCoeffs2=dist03,
    imageSize=image_size,
    R=R,
    T=T,
    flags=cv2.CALIB_ZERO_DISPARITY,
    alpha=0  # 0 = no black borders, 1 = keep all image content
)

print("stereo Rectification resultater:")
print("Q matrix (for 3D reconstruction):\n", Q)

# Build undistort/rectify maps for fast remapping
map02_x, map02_y = cv2.initUndistortRectifyMap(
    mtx02, dist02, R1, P1, image_size, cv2.CV_32FC1
)

map03_x, map03_y = cv2.initUndistortRectifyMap(
    mtx03, dist03, R2, P2, image_size, cv2.CV_32FC1
)

